In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
import spacy
import re
import nltk
from spacy.tokenizer import Tokenizer
import pickle
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [2]:
from nltk import word_tokenize
from sklearn import utils

In [3]:
import csv
import re
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

In [4]:
dt = pd.read_csv('train.csv')

In [5]:
dt.dropna()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1
...,...,...,...,...,...,...
12115,2b78e2a914,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,en,English,2
12116,7e9943d152,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,en,English,0
12117,5085923e6c,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",en,English,2
12118,fc8e2fd1fe,At the west end is a detailed model of the who...,The model temple complex is at the east end.,en,English,2


# English

In [6]:
# english data set
df_en = dt.loc[dt['lang_abv'] == 'en']
df_en

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
7,fdcd1bd867,From Cockpit Country to St. Ann's Bay,From St. Ann's Bay to Cockpit Country.,en,English,2
8,7cfb3d272c,"Look, it's your skin, but you're going to be i...",The boss will fire you if he sees you slacking...,en,English,1
...,...,...,...,...,...,...
12115,2b78e2a914,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,en,English,2
12116,7e9943d152,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,en,English,0
12117,5085923e6c,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",en,English,2
12118,fc8e2fd1fe,At the west end is a detailed model of the who...,The model temple complex is at the east end.,en,English,2


In [7]:
# split train and test
X_train_en, X_test_en= train_test_split(df_en, test_size=0.3, random_state=100)

In [8]:
#create a csv file
X_train_en.to_csv('X_train_en.csv', encoding='utf-8', index = False)
X_test_en.to_csv('X_test_en.csv', encoding='utf-8', index = False)

In [9]:
#load the csv file to create the input data for Doc2vec
data_train = list()
words_train = list()
with open('X_train_en.csv', encoding='utf-8', newline='') as infile:
    reader = csv.reader(infile)
    for row in reader:
        sent = [word_tokenize(row[i].lower()) for i in range(1,3)]
        words_train = sent[0]+ sent[1]
        #print(words)
        data_train.append(TaggedDocument(words_train,[row[5]]))

In [10]:
data_train.pop(0)

TaggedDocument(words=['premise', 'hypothesis'], tags=['label'])

In [12]:
#load the csv file to create the input data for Doc2vec
data_test = list()
words_test = list()
with open('X_test_en.csv', encoding='utf-8', newline='') as infile:
    reader = csv.reader(infile)
    for row in reader:
        sent = [word_tokenize(row[i].lower()) for i in range(1,3)]
        words_test = sent[0]+ sent[1]
        #print(words)
        data_test.append(TaggedDocument(words_test,[row[5]]))

In [13]:
data_test.pop(0)

TaggedDocument(words=['premise', 'hypothesis'], tags=['label'])

In [14]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [15]:
import multiprocessing
import tqdm
cores = multiprocessing.cpu_count()

In [16]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab(data_train)

2022-09-28 11:50:55,993 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d300,n5,mc2,t8)', 'datetime': '2022-09-28T11:50:55.992715', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'created'}
2022-09-28 11:50:55,997 : INFO : collecting all words and their counts
2022-09-28 11:50:55,997 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2022-09-28 11:50:56,030 : INFO : collected 11828 word types and 3 unique tags from a corpus of 4809 examples and 158245 words
2022-09-28 11:50:56,031 : INFO : Creating a fresh vocabulary
2022-09-28 11:50:56,064 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=2 retains 8637 unique words (73.02164355765979%% of original 11828, drops 3191)', 'datetime': '2022-09-28T11:50:56.064366', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.

In [17]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle(data_train), total_examples=len(data_train), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

2022-09-28 11:51:12,415 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 8 workers on 8637 vocabulary and 300 features, using sg=1 hs=0 sample=0 negative=5 window=5', 'datetime': '2022-09-28T11:51:12.415497', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'train'}
2022-09-28 11:51:12,563 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-09-28 11:51:12,564 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-09-28 11:51:12,566 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-09-28 11:51:12,568 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-09-28 11:51:12,568 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-09-28 11:51:12,569 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-09-28 11:51:12,569 : INFO : worker thread finished; awaiting finish o

2022-09-28 11:51:13,355 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-09-28 11:51:13,356 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-09-28 11:51:13,360 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-09-28 11:51:13,361 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-09-28 11:51:13,362 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-09-28 11:51:13,363 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-09-28 11:51:13,365 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-09-28 11:51:13,367 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-09-28 11:51:13,367 : INFO : EPOCH - 1 : training on 158245 raw words (159863 effective words) took 0.1s, 1112621 effective words/s
2022-09-28 11:51:13,368 : INFO : Doc2Vec lifecycle event {'msg': 'training on 158245 raw words (159863 effective words) took 0.2s, 10473

2022-09-28 11:51:14,141 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-09-28 11:51:14,142 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-09-28 11:51:14,143 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-09-28 11:51:14,144 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-09-28 11:51:14,144 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-09-28 11:51:14,145 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-09-28 11:51:14,145 : INFO : EPOCH - 1 : training on 158245 raw words (159863 effective words) took 0.1s, 1167865 effective words/s
2022-09-28 11:51:14,146 : INFO : Doc2Vec lifecycle event {'msg': 'training on 158245 raw words (159863 effective words) took 0.1s, 1109852 effective words/s', 'datetime': '2022-09-28T11:51:14.146102', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windo

2022-09-28 11:51:14,842 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-09-28 11:51:14,842 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-09-28 11:51:14,843 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-09-28 11:51:14,844 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-09-28 11:51:14,845 : INFO : EPOCH - 1 : training on 158245 raw words (159863 effective words) took 0.1s, 1263209 effective words/s
2022-09-28 11:51:14,845 : INFO : Doc2Vec lifecycle event {'msg': 'training on 158245 raw words (159863 effective words) took 0.1s, 1192897 effective words/s', 'datetime': '2022-09-28T11:51:14.845255', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'train'}
2022-09-28 11:51:14,847 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 8 workers on 8637 vocabulary and 300 features, using sg

2022-09-28 11:51:15,522 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-09-28 11:51:15,522 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-09-28 11:51:15,522 : INFO : EPOCH - 1 : training on 158245 raw words (159863 effective words) took 0.1s, 1331251 effective words/s
2022-09-28 11:51:15,523 : INFO : Doc2Vec lifecycle event {'msg': 'training on 158245 raw words (159863 effective words) took 0.1s, 1254232 effective words/s', 'datetime': '2022-09-28T11:51:15.523068', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'train'}
2022-09-28 11:51:15,525 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 8 workers on 8637 vocabulary and 300 features, using sg=1 hs=0 sample=0 negative=5 window=5', 'datetime': '2022-09-28T11:51:15.525067', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'p

2022-09-28 11:51:16,206 : INFO : EPOCH - 1 : training on 158245 raw words (159863 effective words) took 0.1s, 1307524 effective words/s
2022-09-28 11:51:16,206 : INFO : Doc2Vec lifecycle event {'msg': 'training on 158245 raw words (159863 effective words) took 0.1s, 1233030 effective words/s', 'datetime': '2022-09-28T11:51:16.206315', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'train'}
2022-09-28 11:51:16,208 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 8 workers on 8637 vocabulary and 300 features, using sg=1 hs=0 sample=0 negative=5 window=5', 'datetime': '2022-09-28T11:51:16.208308', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'train'}
2022-09-28 11:51:16,323 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-09-28 11:51:16,327 : INFO : w

Wall time: 4.31 s


In [18]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [19]:
y_train, X_train = vec_for_learning(model_dbow, data_train)
y_test, X_test = vec_for_learning(model_dbow, data_test)

In [ ]:
X_train

In [20]:
y_train

('1',
 '2',
 '2',
 '1',
 '0',
 '2',
 '1',
 '0',
 '0',
 '2',
 '1',
 '2',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '2',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '2',
 '0',
 '2',
 '1',
 '1',
 '2',
 '2',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '2',
 '1',
 '2',
 '1',
 '2',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '2',
 '1',
 '2',
 '2',
 '1',
 '1',
 '0',
 '2',
 '1',
 '1',
 '2',
 '0',
 '2',
 '1',
 '1',
 '1',
 '2',
 '0',
 '1',
 '1',
 '1',
 '2',
 '1',
 '1',
 '0',
 '1',
 '1',
 '2',
 '0',
 '0',
 '0',
 '1',
 '2',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '2',
 '2',
 '0',
 '1',
 '0',
 '0',
 '2',
 '1',
 '2',
 '1',
 '0',
 '2',
 '2',
 '0',
 '0',
 '1',
 '0',
 '2',
 '2',
 '0',
 '1',
 '2',
 '1',
 '2',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '2',
 '2',
 '1',
 '1',
 '1',
 '2',
 '1',
 '0',
 '1',
 '2',
 '0',
 '2',
 '2',
 '0',
 '0',
 '0',
 '2',
 '0',
 '2',
 '2',
 '2',
 '2',
 '0',
 '0',
 '2',
 '1',
 '1',
 '1',
 '2',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '2',
 '0',
 '1'

In [21]:
# data preprocessing
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

scaler =  MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Naive Bayes Classifier

In [22]:
model = MultinomialNB()

model.fit(X_train,y_train)
predictions = model.predict(X_test)

print('Classification report:')
print(classification_report(y_test, predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test, predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.26      0.08      0.12       703
           1       0.30      0.30      0.30       667
           2       0.33      0.57      0.42       691

    accuracy                           0.31      2061
   macro avg       0.30      0.32      0.28      2061
weighted avg       0.30      0.31      0.28      2061

Confusion matrix:
[[ 54 248 401]
 [ 83 198 386]
 [ 73 222 396]]


## Linear SVM

In [23]:
# without pipeline
clf = LinearSVC(C=1.0, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.31392527899078115
F1-score [0.36107711 0.24765158 0.3143508 ]
              precision    recall  f1-score   support

           0       0.32      0.42      0.36       703
           1       0.29      0.22      0.25       667
           2       0.33      0.30      0.31       691

    accuracy                           0.31      2061
   macro avg       0.31      0.31      0.31      2061
weighted avg       0.31      0.31      0.31      2061



C:\Users\eleon\anaconda3\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [24]:
#with pipeline and grid serach
param_grid = [{'sel__k': [8, 10, 50,'all'], 'learner__C': [0.1, 0.5, 1, 10]}]

opt_pipeline = Pipeline([
    
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', LinearSVC())  # learning algorithm
])

n_jobs = 3 #Number of jobs to run in parallel
SVM_opt_search = GridSearchCV(opt_pipeline, param_grid, n_jobs = n_jobs, verbose=True).fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [25]:
SVM_opt_search.best_params_

{'learner__C': 0.1, 'sel__k': 50}

In [26]:
opt_predictions = SVM_opt_search.best_estimator_.predict(X_test)

print('Classification report:')
print(classification_report(y_test, opt_predictions, zero_division = 1))
print('Confusion matrix:')
cm = confusion_matrix(y_test, opt_predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.30      0.41      0.34       703
           1       0.27      0.20      0.23       667
           2       0.28      0.25      0.27       691

    accuracy                           0.29      2061
   macro avg       0.28      0.29      0.28      2061
weighted avg       0.28      0.29      0.28      2061

Confusion matrix:
[[285 180 238]
 [330 132 205]
 [346 170 175]]


In [27]:
from sklearn.svm import SVC

In [28]:
clf = clf = SVC(gamma='auto', C=0.1, kernel='rbf', random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.3410965550703542
F1-score [0.50868307 0.         0.        ]
              precision    recall  f1-score   support

           0       0.34      1.00      0.51       703
           1       0.00      0.00      0.00       667
           2       0.00      0.00      0.00       691

    accuracy                           0.34      2061
   macro avg       0.11      0.33      0.17      2061
weighted avg       0.12      0.34      0.17      2061



C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
clf.get_params()

{'C': 0.1,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'auto',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 42,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [29]:
#with pipeline and grid serach
param_grid = [{'sel__k': [8, 10, 50,'all'], 'learner__C': [0.1, 0.5, 1, 10]}]

opt_pipeline = Pipeline([
    
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', SVC(kernel= 'rbf'))  # learning algorithm
])

n_jobs = 3 #Number of jobs to run in parallel
SVM_opt_search = GridSearchCV(opt_pipeline, param_grid, n_jobs = n_jobs, verbose=True).fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [30]:
opt_predictions = SVM_opt_search.best_estimator_.predict(X_test)

print('Classification report:')
print(classification_report(y_test, opt_predictions, zero_division = 1))
print('Confusion matrix:')
cm = confusion_matrix(y_test, opt_predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.31      0.49      0.38       703
           1       0.27      0.15      0.19       667
           2       0.28      0.24      0.26       691

    accuracy                           0.29      2061
   macro avg       0.29      0.29      0.28      2061
weighted avg       0.29      0.29      0.28      2061

Confusion matrix:
[[345 129 229]
 [385  98 184]
 [389 138 164]]


## Random Forest

In [34]:
rd =  RandomForestClassifier(bootstrap = True)
rd.fit(X_train,y_train)
pred = rd.predict(X_test)

print('Classification report:')
print(classification_report(y_test, pred, zero_division = 1))
print('Confusion matrix:')
cm = confusion_matrix(y_test, pred)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.31      0.37      0.34       703
           1       0.30      0.25      0.27       667
           2       0.30      0.30      0.30       691

    accuracy                           0.31      2061
   macro avg       0.30      0.30      0.30      2061
weighted avg       0.30      0.31      0.30      2061

Confusion matrix:
[[258 199 246]
 [268 165 234]
 [299 186 206]]


In [35]:
param_grid = [{'sel__k': [8, 10, 50,'all'], 'learner__n_estimators': [200, 300, 500, 700]}]

opt_pipeline = Pipeline([
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', RandomForestClassifier(bootstrap = True))  # learning algorithm
])

n_jobs = 3 #Number of jobs to run in parallel
opt_search = GridSearchCV(opt_pipeline, param_grid, cv=5, n_jobs = n_jobs, verbose=True).fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [36]:
opt_search.best_params_

{'learner__n_estimators': 300, 'sel__k': 50}

In [37]:
opt_predictions = opt_search.best_estimator_.predict(X_test)

print('Classification report:')
print(classification_report(y_test, opt_predictions, zero_division = 1))
print('Confusion matrix:')
cm = confusion_matrix(y_test, opt_predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.31      0.38      0.34       703
           1       0.29      0.22      0.25       667
           2       0.31      0.31      0.31       691

    accuracy                           0.31      2061
   macro avg       0.30      0.30      0.30      2061
weighted avg       0.30      0.31      0.30      2061

Confusion matrix:
[[266 184 253]
 [293 150 224]
 [294 184 213]]
